5주차 예습과제에서는 6주차 내용인 GPT의 생성 능력을 맛보고자 합니다. 사용할 모델은 koalpaca라는 모델인데, 한국어 버전의 alpaca 모델입니다.

alpaca는 llama라는 모델을 instruction fine-tuning 해서 사람의 지시를 잘 따르는 모델입니다.

llama라는 모델은 LLaMA-13B의 경우 GPT-3보다 10배이상 작음에도 불구하고 대부분의 평가서 GPT-3보다 우수한 성능을 보이며, 더 나아가 LLaMA-65B의 경우 대부분의 벤치마크에서 Chinchilla, Gopher, GPT-3, PaLM와 유사하거나 더 뛰어난 결과를 보이는 훌륭한 모델입니다.

그러면 지금부터 koalpaca로 생성 모델을 한번 경험해보도록 하겠습니다!

In [2]:
# GPU 켜져 있는지 확인

!nvidia-smi

Wed Feb 14 16:28:25 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.154.05             Driver Version: 535.154.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA GeForce RTX 3080 Ti     Off | 00000000:01:00.0  On |                  N/A |
|  0%   58C    P5              33W / 400W |  11197MiB / 12288MiB |     46%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [3]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

/home/jj/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


peft는 parameter efficient fine-tuning의 약자로, 큰 모델을 코랩에서 돌릴 수 있도록 쪼개서 만든 것입니다.

오래 걸립니다!

In [5]:
from peft import PeftModel, PeftConfig

In [6]:
peft_model_id = "beomi/qlora-koalpaca-polyglot-12.8b-50step"
config = PeftConfig.from_pretrained(peft_model_id)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, quantization_config=bnb_config, device_map={"":0})
model = PeftModel.from_pretrained(model, peft_model_id)
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

model.eval()

config.json: 100%|██████████| 686/686 [00:00<00:00, 1.16MB/s]
model.safetensors.index.json: 100%|██████████| 52.5k/52.5k [00:00<00:00, 45.0MB/s]
model-00001-of-00028.safetensors: 100%|██████████| 946M/946M [01:24<00:00, 11.2MB/s]
model-00002-of-00028.safetensors: 100%|██████████| 843M/843M [01:15<00:00, 11.2MB/s]
model-00003-of-00028.safetensors: 100%|██████████| 843M/843M [01:15<00:00, 11.2MB/s]
model-00004-of-00028.safetensors: 100%|██████████| 1.00G/1.00G [01:30<00:00, 11.2MB/s]
model-00005-of-00028.safetensors: 100%|██████████| 896M/896M [01:19<00:00, 11.2MB/s]
model-00006-of-00028.safetensors: 100%|██████████| 1.00G/1.00G [01:31<00:00, 11.0MB/s]
model-00007-of-00028.safetensors: 100%|██████████| 896M/896M [01:20<00:00, 11.1MB/s]
model-00008-of-00028.safetensors: 100%|██████████| 1.00G/1.00G [01:33<00:00, 10.8MB/s]
model-00009-of-00028.safetensors: 100%|██████████| 896M/896M [01:21<00:00, 11.0MB/s]
model-00010-of-00028.safetensors: 100%|██████████| 1.00G/1.00G [01:35<00:00, 10.5MB/

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(30080, 5120)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-39): 40 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=5120, out_features=15360, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                

next token generation

In [7]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 수수께끼에 도전한다. 그는 인공지능 분야의 최고 권위자로, MIT 인공지능연구소 소장을 역임했다. IBM 연구원으로 인공 지능 및 시스템 신경망 분야 연구를 선도했으며, 이후 하버드대학교


 QA

In [8]:
def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [9]:
gen('건강하게 살기 위한 세 가지 방법은?')

/home/jj/.local/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:447: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: ① 마음을 편안하게 다스려야 건강해진다. 우리의 인체는 자율신경에 의해 조절되는 ‘기(氣)’로 구성되어 있어서, 사람이 분노를 하면 그 기운이 머리 쪽으로 올라가는데 이것


top_k와 같은 parameter를 조절하면서 실험해보세요!

In [10]:
prompt = '인간처럼 생각하고, 행동하는 \'지능\'을 통해 인류가 이제까지 풀지 못했던'
with torch.no_grad():
  tokens = tokenizer.encode(prompt, return_tensors='pt').to(device='cuda', non_blocking=True)
  gen_tokens = model.generate(tokens, do_sample=True, temperature=0.8, max_length=64, top_k=50)
  generated = tokenizer.batch_decode(gen_tokens)[0]

print(generated)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


인간처럼 생각하고, 행동하는 '지능'을 통해 인류가 이제까지 풀지 못했던 문제를 해결하는 것이 가능해진다면, 우리는 그것을 인류의 진화라고 부를 수 있고, 또 그렇게 불러야 마땅하다. 인공지능 연구의 역사 인공지능의 역사를 돌이켜보면
